# Do in-hand services deter churn more than low cost?

## Goal: 
    - discover drivers of churn from telco dataset
    - use machine learning to classify cutomers as likley or unlikely to churn based on drivers
    - To answer the question, to prevent churn do we need to sell more products or focus on retention? And if we need to sell more products, does the accessibility of the product matter?

## Data Dictionary:

## Imports:

In [1]:
import pydataset as data
import pandas as pd
import numpy as np
import seaborn as sns
import math
import os

from scipy import stats
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import env
import wrangle
import acquire 

## Acquire:

In [2]:
df = acquire.new_telco_data()

## Prepare:

## A brief look at the data:

## A summary of the data:

# Explore:

### Question 1: Does having multiple in-hand services affect churn?

In [ ]:
#baseline for churn
df_3.churn.value_counts(normalize=True)

In [ ]:
#code for viz

explaination: stating what the viz shows

### Question 2: Is churn correlated to the size of a household?

In [ ]:
#code for viz

explaination: stating what the viz shows

### Question 3: Does an increase in monthly charges affect churn?

In [ ]:
#code for viz

explaination: stating what the viz shows

### Question 4: Is there a "safety tenure?"

In [ ]:
#code for viz

explaination: stating what the viz shows

### Question 5: if there is a safety tenure, is in-hand service amount correlated?

In [ ]:
#code for viz

explaination: stating what the viz shows

### Question 6: Is churn driven by a lack of a certain individual product?

In [ ]:
#code for viz

explaination: stating what the viz shows

# Exploration Summary:

# Features I am moving to Modeling:

# Modeling:

### Model 1

### Model 2

### Model 3

### Model 4

## Comparing models

# Model on Test Set

### Modeling Summary:

# Conclusions:

### Exploration:

### Modeling:

### Recommendations:

### Next Steps: